# Model Building - Random Forest

This notebook implements the model building for my thesis comparing different supervised machine learning techniques (logistic regression, random forests, support vector machines and neural networks). The main focus surrounds when each method results in better performance as measured by discrimination (AUC) and calibration (calibration plots). This project considers various factors including the number of features, number of datapoints and 'predictor strength'. This notebook fits random forests to a small number of variables and outputs prediction and results to a pandas dataframe, where model evaliation and results will be undertaken in a following notebook.

In [1]:
import glob
import numpy as np
import os
import pandas as pd
import sklearn

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, auc
from sklearn.model_selection import StratifiedKFold, train_test_split
from scipy.stats import loguniform, uniform, randint

np.random.seed(21)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
def hyperparameter_randomiser_rf(max_depth_range,
                                 max_features_range,
                                 min_leaf_range,
                                 n_trees_range,
                                 prints = False):
    '''
    Returns the best hyperparameters from given ranges using a random search algorithm 
    (all searches use uniform distribution):
    max_depth_range: Range of max_depth
    max_features_range: Range of maximum features per split
    min_leaf_range: Range of minimum data points per leaf node
    n_trees_range: Range of number of trees in a constituent random forest
    prints: An option to facilitate the printing of the randomly selected hyperparameters at each search.
    '''
    max_depth = randint(max_depth_range[0], max_depth_range[1] + 1).rvs(1).item()
    max_features = randint(max_features_range[0], max_features_range[1] + 1).rvs(1).item()
    min_leaf = randint(min_leaf_range[0], min_leaf_range[1] + 1).rvs(1).item()
    n_trees = randint(n_trees_range[0], n_trees_range[1] + 1).rvs(1).item()
    if prints == True:
        print("max_depth:", max_depth)
        print("max_features:", max_features)
        print("min_leaf:", min_leaf)
        print("n_trees:", n_trees)
    return max_depth, max_features, min_leaf, n_trees

In [3]:
def build_model_rf(max_depth, max_features, min_leaf, n_trees):
    rf = RandomForestClassifier(n_estimators = n_trees, 
                                max_depth = max_depth, 
                                max_features = max_features,
                                min_samples_leaf = min_leaf)
    return rf

In [4]:
def hyp_tuner_rf(md_range, 
                 mf_range,
                 ml_range,
                 nt_range,
                 X_train, 
                 y_train, 
                 X_test, 
                 y_test,
                 cv_folds = 5, 
                 random_search = 20):
    '''Returns the best hyperparameters from given ranges using a random search algorithm:
        md_range: Range of max depth of consituent decision trees
        mf_range: Range of max features in each split
        X_train: The training dataset (as a pandas dataframe) features
        y_train: The training dataset (as a pandas dataframe) labels
        X_test: The testing dataset (as a pandas dataframe) features
        y_test: The testing dataset (as a pandas dataframe) labels
        cv_folds: Number of folds of cross validation (default 5)
        random_search: how many random searches are performed (default 20)
        '''
    cv = StratifiedKFold(n_splits = cv_folds, shuffle = True, random_state = 21)
    cv.get_n_splits(X_train, y_train)
    #splitting the dataset for k-fold
    md_list, mf_list, ml_list, nt_list = [], [], [], []
    auc_dict = dict()
    #creating dictionaries and lists that store information on hyperparameters and aucs
    for i in range(random_search):
        print("Running search {counter} out of {maxi}".format(counter = str(i+1), maxi = str(random_search)))
        #randomly selecting hyperparameters and storing them
        md, mf, ml, nt = hyperparameter_randomiser_rf(max_depth_range = md_range,
                                                      max_features_range = mf_range,
                                                      min_leaf_range = ml_range,
                                                      n_trees_range = nt_range) 
        md_list.append(md)
        mf_list.append(mf)
        ml_list.append(ml)
        nt_list.append(nt)
        auc_dict[i] = []
        for train_index, test_index in cv.split(X_train, y_train):
            #cross validating
            model = build_model_rf(max_depth = md,
                                   max_features = mf,
                                   min_leaf = ml,
                                   n_trees = nt)
            X1_train, X1_valid = X_train.iloc[train_index], X_train.iloc[test_index]
            y1_train, y1_valid = y_train.iloc[train_index], y_train.iloc[test_index]
            model.fit(X1_train, y1_train)
            y1_pred = model.predict_proba(X1_valid)[:, 1]
            fpr, tpr, threshold = roc_curve(y1_valid, y1_pred)
            cauc = auc(fpr, tpr)
            auc_dict[i].append(cauc)
    #analysing results to find best hyperparameters
    hp_dict = {"max_depth": md_list,
               "max_features": mf_list,
               "min_leaf": ml_list,
               "n_trees": nt_list}
    av_auc_dict = dict()
    for k, v in auc_dict.items():
        av_auc_dict[k] = sum(v)/len(v)
    opt_hyper = max(av_auc_dict, key = av_auc_dict.get)
    opt_md = hp_dict["max_depth"][opt_hyper]
    opt_mf = hp_dict["max_features"][opt_hyper]
    opt_ml = hp_dict["min_leaf"][opt_hyper]
    opt_nt = hp_dict["n_trees"][opt_hyper]
    #building the model based on the whole training data based on optimal parameters
    opt_model = build_model_rf(max_depth = opt_md, 
                               max_features = opt_mf,
                               min_leaf = opt_ml,
                               n_trees = opt_nt)
    opt_model.fit(X_train, y_train)
    y_pred = opt_model.predict_proba(X_test)[:, 1]
    v_auc = roc_auc_score(y_test, y_pred)
    #sorting prediction df
    y_test_df = pd.DataFrame(y_test)
    pred_df = pd.DataFrame(y_pred)
    pred_df['index'] = y_test.index
    pred_df.set_index('index', inplace = True)
    pred_df.rename(columns = {0: "prob"}, inplace = True)
    df = pred_df.join(y_test_df, how = 'left')
    print("The best hyperparameters were:")
    print("max_depth:", opt_md)
    print("max_features:", opt_mf)
    print("min_leaf:", opt_ml)
    print("n_trees:", opt_nt)
    y_pred2 = opt_model.predict(X_test)
    print(confusion_matrix(y_test, y_pred2))
    
    return df, v_auc

In [5]:
current_wd = os.getcwd()
base_wd = current_wd
base_wd = base_wd.replace("\\", "/")
path = base_wd + "/data/bootstraps"
os.chdir(path)
all_filenames = [i for i in glob.glob('*.{}'.format('csv'))]

In [6]:
%%time
#fitting random_forests to each bootstrap
np.random.seed(21)
i = 0
dataframes = []
for filename in all_filenames:
    i += 1
    print(str(i) + " of " + str(len(all_filenames)))
    mf = pd.read_csv(filename)
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(mf.drop('deceased', axis = 1), 
                                                                        mf['deceased'], test_size = 0.2, 
                                                                        random_state = 21, 
                                                                        stratify = mf['deceased'])
    preds, rf_auc = hyp_tuner_rf(md_range = [3, 8],
                                 mf_range = [1, 9],
                                 ml_range = [1, 5],
                                 nt_range = [100, 1000],
                                 X_train = X_train,
                                 y_train = y_train,
                                 X_test = X_test,
                                 y_test = y_test, 
                                 cv_folds = 3,
                                 random_search = 5)
    preds.reset_index(drop = True, inplace = True)
    dataframes.append(preds)
    print(rf_auc)
rf_predictions = pd.concat(dataframes, axis = 1)
rf_predictions.to_csv('./predictions/rf_predictions.csv', index = False)

1 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
max_depth: 6
max_features: 4
min_leaf: 1
n_trees: 383
[[806  19]
 [ 76  51]]
0.917642567406347
2 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
max_depth: 7
max_features: 7
min_leaf: 4
n_trees: 975
[[806  11]
 [ 75  60]]
0.8965682941203137
3 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
max_depth: 8
max_features: 9
min_leaf: 1
n_trees: 558
[[798  23]
 [ 64  67]]
0.9109817667897091
4 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
max_depth: 8
max_features: 8
min_

Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
max_depth: 7
max_features: 9
min_leaf: 1
n_trees: 721
[[809  12]
 [ 74  57]]
0.9219440079590149
33 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
max_depth: 8
max_features: 2
min_leaf: 3
n_trees: 968
[[795  21]
 [ 84  52]]
0.9185319348327566
34 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
max_depth: 7
max_features: 6
min_leaf: 5
n_trees: 268
[[810  14]
 [ 70  58]]
0.9152855734223301
35 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
max_depth: 4
max_features: 6
min_leaf: 5
n_trees: 169
[[789  31]

Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
max_depth: 8
max_features: 3
min_leaf: 1
n_trees: 305
[[805  18]
 [ 66  63]]
0.9378149519153787
64 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
max_depth: 8
max_features: 5
min_leaf: 2
n_trees: 755
[[809  16]
 [ 64  63]]
0.9343736578382248
65 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
max_depth: 6
max_features: 5
min_leaf: 2
n_trees: 316
[[809  12]
 [ 76  55]]
0.8953240788091232
66 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
max_depth: 7
max_features: 8
min_leaf: 1
n_trees: 197
[[793  20]

Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
max_depth: 8
max_features: 8
min_leaf: 4
n_trees: 404
[[803  16]
 [ 55  78]]
0.9449998622930954
95 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
max_depth: 8
max_features: 3
min_leaf: 2
n_trees: 619
[[800  12]
 [ 81  59]]
0.9270056298381422
96 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
max_depth: 8
max_features: 6
min_leaf: 1
n_trees: 909
[[801  20]
 [ 69  62]]
0.9053472306161728
97 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
max_depth: 7
max_features: 8
min_leaf: 1
n_trees: 134
[[797  17]

Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
max_depth: 8
max_features: 4
min_leaf: 2
n_trees: 900
[[825   1]
 [ 62  64]]
0.9343940966216995
126 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
max_depth: 6
max_features: 9
min_leaf: 2
n_trees: 448
[[793  27]
 [ 74  58]]
0.8926182557280118
127 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
max_depth: 8
max_features: 4
min_leaf: 3
n_trees: 956
[[805  15]
 [ 72  60]]
0.9314486326681448
128 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
max_depth: 8
max_features: 7
min_leaf: 2
n_trees: 303
[[816  

...

Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
max_depth: 7
max_features: 4
min_leaf: 2
n_trees: 412
[[813  11]
 [ 77  51]]
0.9315837378640777
870 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
max_depth: 8
max_features: 8
min_leaf: 3
n_trees: 405
[[790  23]
 [ 72  67]]
0.9164211066571099
871 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
max_depth: 8
max_features: 3
min_leaf: 2
n_trees: 316
[[802  18]
 [ 75  57]]
0.9286770140428677
872 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
max_depth: 6
max_features: 5
min_leaf: 1
n_trees: 674
[[812  

Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
max_depth: 7
max_features: 7
min_leaf: 1
n_trees: 371
[[796  23]
 [ 71  62]]
0.90670816234726
901 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
max_depth: 6
max_features: 7
min_leaf: 2
n_trees: 423
[[807  22]
 [ 70  53]]
0.9130404934929929
902 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
max_depth: 8
max_features: 6
min_leaf: 5
n_trees: 414
[[808  13]
 [ 69  62]]
0.9177506485295348
903 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
max_depth: 8
max_features: 2
min_leaf: 2
n_trees: 276
[[808  12

Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
max_depth: 8
max_features: 1
min_leaf: 1
n_trees: 435
[[821   3]
 [ 80  48]]
0.9322474211165048
932 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
max_depth: 7
max_features: 9
min_leaf: 1
n_trees: 153
[[810  19]
 [ 63  60]]
0.9039689311247756
933 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
max_depth: 8
max_features: 4
min_leaf: 5
n_trees: 120
[[808  14]
 [ 72  58]]
0.9035841287666105
934 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
max_depth: 8
max_features: 8
min_leaf: 1
n_trees: 549
[[818  

Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
max_depth: 8
max_features: 5
min_leaf: 1
n_trees: 919
[[812  11]
 [ 64  65]]
0.9324648902201249
963 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
max_depth: 8
max_features: 7
min_leaf: 3
n_trees: 388
[[802  20]
 [ 59  71]]
0.9109582631480442
964 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
max_depth: 8
max_features: 1
min_leaf: 5
n_trees: 715
[[803  11]
 [ 94  44]]
0.9094826051347791
965 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
max_depth: 8
max_features: 3
min_leaf: 3
n_trees: 759
[[807  

Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
max_depth: 8
max_features: 2
min_leaf: 2
n_trees: 795
[[796  13]
 [ 90  53]]
0.9194464373697995
994 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
max_depth: 8
max_features: 1
min_leaf: 1
n_trees: 381
[[814   8]
 [ 86  44]]
0.9196238068500843
995 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
max_depth: 7
max_features: 7
min_leaf: 1
n_trees: 660
[[801  18]
 [ 59  74]]
0.919762776905634
996 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
max_depth: 8
max_features: 3
min_leaf: 2
n_trees: 980
[[809  1